In [0]:
%run ../Includes/Copy-Datasets

In [0]:
%sql
SELECT COUNT(*)
FROM orders_silver
WHERE order_timestamp >= '2020-01-01'

count(1)
4000


Check and see if all orders have a timestamp greater than 2020-01-01

In [0]:
%sql
ALTER TABLE orders_silver ADD CONSTRAINT timestamp_within_range CHECK (order_timestamp >= '2020-01-01');

Look at the constraint under table properties 

In [0]:
%sql
DESCRIBE EXTENDED orders_silver

col_name,data_type,comment
order_id,string,null
order_timestamp,timestamp,null
customer_id,string,null
quantity,bigint,null
total,bigint,null
books,array>,null
,,
# Delta Statistics Columns,,
Column Names,"quantity, order_id, order_timestamp, customer_id, total, books",
Column Selection Method,first-32,


The following job will fail because the values we are trying to add into the table violate the constraint we have just set. 

In [0]:
%sql
INSERT INTO orders_silver
VALUES ('1', '2022-02-01 00:00:00.000', 'C00001', 0, 0, NULL),
       ('2', '2019-05-01 00:00:00.000', 'C00001', 0, 0, NULL),
       ('3', '2023-01-01 00:00:00.000', 'C00001', 0, 0, NULL)

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: com.databricks.sql.transaction.tahoe.schema.InvariantViolationException: CHECK constraint timestamp_within_range (order_timestamp >= '2020-01-01') violated by row with values:
 - order_timestamp : 2019-05-01 00:00:00
	at 0x5366c7c .CreateError(external/workspace_spark_3_4/photon/exec-nodes/invariant-checker-node.cc:136)
	at 0x53655e4 .NextImpl(external/workspace_spark_3_4/photon/exec-nodes/invariant-checker-node.cc:230)
	at 0x527ae3d .Next(external/workspace_spark_3_4/photon/exec-nodes/exec-node.cc:179)
	at 0x53386cf .OpenImpl(external/workspace_spark_3_4/photon/exec-nodes/file-writer-node.cc:179)
	at com.databricks.photon.JniApiImpl.open(Native Method)
	at com.databricks.photon.JniApi.open(JniApi.scala)
	at com.databricks.photon.JniExecNode.open(JniExecNode.java:75)
	at com.databricks.photon.PhotonWriteResultHandler.$anonfun$getResult$3(PhotonWriteStageExec.scala:121)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.photon.PhotonResultHandler.timeit(PhotonResultHandler.scala:30)
	at com.databricks.photon.PhotonResultHandler.timeit$(PhotonResultHandler.scala:28)
	at com.databricks.photon.PhotonWriteResultHandler.timeit(PhotonWriteStageExec.scala:67)
	at com.databricks.photon.PhotonWriteResultHandler.$anonfun$getResult$2(PhotonWriteStageExec.scala:121)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1810)
	at com.databricks.photon.PhotonWriteResultHandler.getResult(PhotonWriteStageExec.scala:118)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.hasNext(PhotonBasicEvaluatorFactory.scala:212)
	at com.databricks.photon.CloseableIterator$$anon$10.hasNext(CloseableIterator.scala:211)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$writeUsingPhoton$1(FileFormatWriter.scala:635)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:82)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:82)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:62)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:196)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:181)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:146)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:129)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:146)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:99)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$8(Executor.scala:900)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1776)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:903)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:798)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)

	at com.databricks.backend.daemon.driver.DriverLocal.executeSql(DriverLocal.scala:387)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.repl(JupyterDriverLocal.scala:713)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$37(DriverLocal.scala:1043)
	at com.databricks.unity.EmptyHandle$.runWith(UCSHandle.scala:128)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$26(DriverLocal.scala:1034)
	at com.databricks.logging.U

Note that if an execution fails, ALL of the records won't be added to the table, even the ones that don't violate any constraints - verify this:

In [0]:
%sql
SELECT *
FROM orders_silver
WHERE order_id IN ('1', '2', '3')

order_id,order_timestamp,customer_id,quantity,total,books


In [0]:
%sql
ALTER TABLE orders_silver ADD CONSTRAINT valid_quantity CHECK (quantity > 0);

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: com.databricks.sql.transaction.tahoe.DeltaAnalysisException: 80 rows in hive_metastore.bookstore_eng_pro.orders_silver violate the new CHECK constraint (quantity > 0)
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.newCheckConstraintViolated(DeltaErrors.scala:320)
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.newCheckConstraintViolated$(DeltaErrors.scala:319)
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.newCheckConstraintViolated(DeltaErrors.scala:3071)
	at com.databricks.sql.transaction.tahoe.commands.AlterTableAddConstraintDeltaCommand.$anonfun$run$38(alterDeltaTableCommands.scala:1060)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.sql.acl.CheckPermissions$.$anonfun$trusted$2(CheckPermissions.scala:2057)
	at com.databricks.sql.util.ThreadLocalTagger.withTag(QueryTagger.scala:62)
	at com.databricks.sql.util.ThreadLocalTagger.withTag$(QueryTagger.scala:59)
	at com.databricks.sql.util.QueryTagger$.withTag(QueryTagger.scala:147)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:2057)
	at com.databricks.sql.transaction.tahoe.commands.AlterTableAddConstraintDeltaCommand.$anonfun$run$37(alterDeltaTableCommands.scala:1054)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.withOperationTypeTag(DeltaLogging.scala:240)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.withOperationTypeTag$(DeltaLogging.scala:227)
	at com.databricks.sql.transaction.tahoe.commands.AlterTableAddConstraintDeltaCommand.withOperationTypeTag(alterDeltaTableCommands.scala:1001)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.$anonfun$recordDeltaOperationInternal$2(DeltaLogging.scala:165)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordFrameProfile(DeltaLogging.scala:309)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordFrameProfile$(DeltaLogging.scala:307)
	at com.databricks.sql.transaction.tahoe.commands.AlterTableAddConstraintDeltaCommand.recordFrameProfile(alterDeltaTableCommands.scala:1001)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.$anonfun$recordDeltaOperationInternal$1(DeltaLogging.scala:164)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:571)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:667)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:685)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:196)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:27)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:470)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:455)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:27)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:662)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:580)
	at com.databricks.spark.util.PublicDBLogging.recordOperationWithResultTags(DatabricksSparkUsageLogger.scala:27)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:571)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:540)
	

In [0]:
%sql
DESCRIBE EXTENDED orders_silver

col_name,data_type,comment
order_id,string,null
order_timestamp,timestamp,null
customer_id,string,null
quantity,bigint,null
total,bigint,null
books,array>,null
,,
# Delta Statistics Columns,,
Column Names,"quantity, order_id, order_timestamp, customer_id, total, books",
Column Selection Method,first-32,


We need to make sure that the rows that violate our constraint are first removed. We take a look at what these rows are:

In [0]:
%sql
SELECT *
FROM orders_silver
where quantity <= 0

order_id,order_timestamp,customer_id,quantity,total,books
000000004650,2022-02-20T11:03:00Z,C00419,0,0,List()
000000004150,2022-01-09T17:01:00Z,C00204,0,0,List()
000000004750,2022-02-28T04:06:00Z,C00403,0,0,List()
000000004800,2022-03-03T10:02:00Z,C00420,0,0,List()
000000003700,2021-12-04T13:09:00Z,C00076,0,0,List()
000000003950,2021-12-25T00:08:00Z,C00126,0,0,List()
000000004200,2022-01-14T03:08:00Z,C00225,0,0,List()
000000004300,2022-01-21T18:02:00Z,C00275,0,0,List()
000000004600,2022-02-16T08:05:00Z,C00376,0,0,List()
000000005050,2022-03-22T23:03:00Z,C00495,0,0,List()


One way to deal with this is to filter out the data that violates the constraints out before streaming to the silver table from the bronze layer. Then, the constraints can be set on the silver table without issue

In [0]:
from pyspark.sql import functions as F

json_schema = "order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>"

query = (spark.readStream.table("bronze")
        .filter("topic = 'orders'")
        .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
        .select("v.*")
        .filter("quantity > 0")
     .writeStream
        .option("checkpointLocation", "dbfs:/mnt/demo_pro/checkpoints/orders_silver")
        .trigger(availableNow=True)
        .table("orders_silver"))

query.awaitTermination()

How to remove a constraint:

In [0]:
%sql
ALTER TABLE orders_silver DROP CONSTRAINT timestamp_within_range;

In [0]:
%sql
DESCRIBE EXTENDED orders_silver

col_name,data_type,comment
order_id,string,null
order_timestamp,timestamp,null
customer_id,string,null
quantity,bigint,null
total,bigint,null
books,array>,null
,,
# Delta Statistics Columns,,
Column Names,"quantity, order_id, order_timestamp, customer_id, total, books",
Column Selection Method,first-32,


We will do more transformations to the data before streaming, so we will drop the table here and then delete the checkpoint. Not recommended in practice 

In [0]:
%sql
DROP TABLE orders_silver

In [0]:
%sql
SELECT * FROM orders_silver LIMIT 20

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `orders_silver` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'GlobalLimit 20
+- 'LocalLimit 20
 +- 'Project [*]
 +- 'UnresolvedRelation [orders_silver], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.tableNotFound(package.scala:90)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2(CheckAnalysis.scala:218)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2$adapted(CheckAnalysis.scala:191)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:248)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:247)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:247)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:247)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:247)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis0(CheckAnalysis.scala:191)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis0$(CheckAnalysis.scala:173)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis0(Analyzer.scala:363)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:169)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:159)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:159)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:363)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$2(Analyzer.scala:418)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker

In [0]:
dbutils.fs.rm("dbfs:/mnt/demo_pro/checkpoints/orders_silver", True)

True